# Тест Параметрические критерии
# курс 4 неделя 2

In [1]:
import numpy as np
import pandas as pd
from scipy import stats
import scipy

## Вопрос 3: 
### о мифе про зевоту

In [2]:
n = 50
n_eks = 34
n_kontr = 16 # равно n - n_eks

n_zev_eks = 10
n_zev_kontr = 4

In [3]:
p_eks = float(n_zev_eks) / n_eks
p_eks

0.29411764705882354

In [4]:
p_kontr = float(n_zev_kontr) / n_kontr
p_kontr

0.25

In [5]:
diff_p = p_eks - p_kontr
diff_p

0.04411764705882354

$H_0$: $p_1$ = $p_2$

$H_1$: $p_1$ > $p_2$

выборки не связанные

$$Z-статистика: Z({X_1, X_2}) =  \frac{\hat{p}_1 - \hat{p}_2}{\sqrt{P(1 - P)(\frac{1}{n_1} + \frac{1}{n_2})}}$$
$$P = \frac{\hat{p}_1{n_1} + \hat{p}_2{n_2}}{{n_1} + {n_2}} $$

In [6]:
def proportions_diff_z_stat_ind(p1, p2, n1, n2):
   
    P = float(p1*n1 + p2*n2) / (n1 + n2)
    
    return (p1 - p2) / np.sqrt(P * (1 - P) * (1. / n1 + 1. / n2))

In [7]:
def proportions_diff_z_test(z_stat, alternative = 'two-sided'):
    if alternative not in ('two-sided', 'less', 'greater'):
        raise ValueError("alternative not recognized\n"
                         "should be 'two-sided', 'less' or 'greater'")
    
    if alternative == 'two-sided':
        return 2 * (1 - scipy.stats.norm.cdf(np.abs(z_stat)))
    
    if alternative == 'less':
        return scipy.stats.norm.cdf(z_stat)

    if alternative == 'greater':
        return 1 - scipy.stats.norm.cdf(z_stat)

In [8]:
print ("p-value: %f" % round(proportions_diff_z_test(proportions_diff_z_stat_ind(p_eks, p_kontr, n_eks, n_kontr),'greater'), 4))

p-value: 0.372900


## вопрос 4:

### банкноты

In [9]:
data = pd.read_csv('data/banknotes.txt', sep='\t')

In [10]:
data.head()

,X1,X2,X3,X4,X5,X6,real
0,214.8,131.0,131.1,9.0,9.7,141.0,1
1,214.6,129.7,129.7,8.1,9.5,141.7,1
2,214.8,129.7,129.7,8.7,9.6,142.2,1
3,214.8,129.7,129.6,7.5,10.4,142.0,1
4,215.0,129.6,129.7,10.4,7.7,141.8,1


In [11]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200 entries, 0 to 199
Data columns (total 7 columns):
X1      200 non-null float64
X2      200 non-null float64
X3      200 non-null float64
X4      200 non-null float64
X5      200 non-null float64
X6      200 non-null float64
real    200 non-null int64
dtypes: float64(6), int64(1)
memory usage: 11.0 KB


In [12]:
from sklearn import cross_validation, linear_model, metrics

C:\ProgramData\Anaconda2\lib\site-packages\sklearn\cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [13]:
X = data.drop('real', 1)
y = data['real']

In [14]:
X_train, X_test, y_train, y_test = cross_validation.train_test_split(X, y, test_size=50, random_state=1)

In [15]:
LR_123 = linear_model.LogisticRegression(random_state=1)
LR_456 = linear_model.LogisticRegression(random_state=1)

проверяем критерием Стьюдента гипотезу равенства двух предсказателей против их неравенства

In [16]:
y_pred_123 = LR_123.fit(X_train[['X1', 'X2', 'X3']], y_train).predict(X_test[['X1', 'X2', 'X3']])

In [17]:
y_pred_456 = LR_123.fit(X_train[['X4', 'X5', 'X6']], y_train).predict(X_test[['X4', 'X5', 'X6']])

In [18]:
print('score for 123-features: %f \nscore for 456-features: %f' 
      % (metrics.accuracy_score(y_test, y_pred_123), metrics.accuracy_score(y_test, y_pred_456)))

score for 123-features: 0.800000 
score for 456-features: 0.980000


In [19]:
print('score for err 123-features: %f \nscore for err 456-features: %f' 
      % ((y_test != y_pred_123).mean(), (y_test != y_pred_456).mean()))

score for err 123-features: 0.200000 
score for err 456-features: 0.020000


In [35]:
def proportions_diff_z_stat_rel(sample1, sample2):
    sample = zip(sample1, sample2)
    n = len(sample)
    
    f = sum([1 if (x[0] == 1 and x[1] == 0) else 0 for x in sample])
    g = sum([1 if (x[0] == 0 and x[1] == 1) else 0 for x in sample])
    
    return float(f - g) / np.sqrt(f + g - float((f - g)**2) / n )

In [36]:
z_stat = proportions_diff_z_stat_rel((y_test != y_pred_123), (y_test != y_pred_456))
z_stat

2.9386041680175268

In [22]:
def proportions_diff_z_test(z_stat, alternative = 'two-sided'):
    if alternative not in ('two-sided', 'less', 'greater'):
        raise ValueError("alternative not recognized\n"
                         "should be 'two-sided', 'less' or 'greater'")
    
    if alternative == 'two-sided':
        return 2 * (1 - scipy.stats.norm.cdf(np.abs(z_stat)))
    
    if alternative == 'less':
        return scipy.stats.norm.cdf(z_stat)

    if alternative == 'greater':
        return 1 - scipy.stats.norm.cdf(z_stat)

In [23]:
print( 'pvalue:', proportions_diff_z_test(z_stat, alternative = 'two-sided'))

('pvalue:', 0.0032969384555543435)


## вопрос 5

 $X_1$ \ $X_2$ | 1| 0 | $\sum$
  ------------- | -------------|
  1  | e | f | e + f
  0  | g | h | g + h
  $\sum$ | e + g| f + h | n  

$$\text{Доверительный интервал для }p_1 - p_2\colon \;\;  \frac{f - g}{n} \pm z_{1-\frac{\alpha}{2}}\sqrt{\frac{f + g}{n^2} - \frac{(f - g)^2}{n^3}}$$

In [24]:
def proportions_diff_confint_rel(sample1, sample2, alpha = 0.05):
    z = scipy.stats.norm.ppf(1 - alpha / 2.)
    sample = zip(sample1, sample2)
    n = len(sample1)
        
    f = sum([1 if (x[0] == 1 and x[1] == 0) else 0 for x in sample])
    g = sum([1 if (x[0] == 0 and x[1] == 1) else 0 for x in sample])
    
    left_boundary = float(f - g) / n  - z * np.sqrt(float((f + g)) / n**2 - float((f - g)**2) / n**3)
    right_boundary = float(f - g) / n  + z * np.sqrt(float((f + g)) / n**2 - float((f - g)**2) / n**3)
    return (left_boundary, right_boundary)

In [25]:
confint = proportions_diff_confint_rel((y_test != y_pred_123), (y_test != y_pred_456), alpha = 0.05)
confint

(0.059945206279614305, 0.30005479372038568)

In [26]:
round(confint[0], 4)

0.0599

## вопрос 6
### экзамен GMAT

In [28]:
mean = 525
std = 100

n_st = 100
mean_st = 541.4

In [29]:
z_stat_6 = (mean_st - mean) / (std / np.sqrt(n_st))

In [31]:
print 'p_value: %f' % round(proportions_diff_z_test(z_stat_6, alternative = 'greater'), 4)

p_value: 0.050500


## вопрос 7

In [32]:
mean_st_new = 541.5

In [33]:
z_stat_7 = (mean_st_new - mean) / (std / np.sqrt(n_st))

In [34]:
print 'p_value: %f' % round(proportions_diff_z_test(z_stat_7, alternative = 'greater'), 4)

p_value: 0.049500
